## Imports

In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow.python.keras.callbacks import EarlyStopping

tf.random.set_seed(0)

# Model Architecture

CNN

In [2]:
# Define the neural network architecture
model = tf.keras.Sequential([
    tf.keras.layers.Reshape((8, 8, 13), input_shape=(832,)),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='linear')
])

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)  # Try different learning rates
model.compile(optimizer=optimizer, loss='MeanSquaredError', metrics=['MeanAbsoluteError'])

### Data Loader

Only bring in data loader if we get more stockfish data, I doubt we will need it...

# Fitting Model

Need to rerun the processedDataset, such that the evaluations are correct.

In [3]:
df = pd.DataFrame(
    np.load("../data/createdData/npyFiles/preprocessedChunk0.npy"),
    np.load("../data/createdData/npyFiles/stockFishEvalsChunk0.npy")
).sample(frac = 1)

X = df.iloc[:, :-1].to_numpy().astype(np.int8)
y = df.iloc[:, -1].to_numpy().astype(np.int16)

In [4]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(X, y, epochs=25, batch_size = 128, validation_split = 0.2, callbacks=[early_stopping])

model.save("../saved_models/stockfish_1mil_25epoch_128batch_0.001learningRate")

Epoch 1/25
6250/6250 [==============================] - 41s 6ms/step - loss: 115501.2891 - mean_absolute_error: 189.9088 - val_loss: 103057.8984 - val_mean_absolute_error: 182.4695
Epoch 2/25
6249/6250 [============================>.] - ETA: 0s - loss: 97816.8594 - mean_absolute_error: 178.4449